In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
REMOVE_ONLY_2_Outliers = True
USE_DUMMY_CAT_FEATURES = True
STACK_MODELLING = True
RANDOM_SEED = 42

In [3]:
df_train = pd.read_csv('./data/train.csv')
df_test =  pd.read_csv('./data/test.csv')

In [4]:
#Remove Id field as it is not useful in regression 
df_test_id = df_test['Id']
#print(df_test_id)

df_train.drop(['Id'],axis=1,inplace=True)
df_test.drop(['Id'],axis=1,inplace=True)

#df_train.drop(['Id','PoolQC','MiscFeature','Alley'],axis=1,inplace=True)
#df_test.drop(['Id','PoolQC','MiscFeature','Alley'],axis=1,inplace=True)

print(df_train.shape)
print(df_test.shape)

(1460, 80)
(1459, 79)


In [5]:
# Remove outliers  Reference - https://www.kaggle.com/zoupet/neural-network-model-for-house-prices-tensorflow
from sklearn.ensemble import IsolationForest

def RemoveOutliers(df_train):
    if(REMOVE_ONLY_2_Outliers):
        #Deleting outliers
        df_train = df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index)
    else:
        clf = IsolationForest(max_samples = 100, random_state = 42)
        clf.fit(df_train)
        y_noano = clf.predict(df_train)
        y_noano = pd.DataFrame(y_noano, columns = ['Top'])
        y_noano[y_noano['Top'] == 1].index.values

        df_train = df_train.iloc[y_noano[y_noano['Top'] == 1].index.values]
        df_train.reset_index(drop = True, inplace = True)
        print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
        print("Number of rows without outliers:", df_train.shape[0])
    return df_train
        
df_train = RemoveOutliers(df_train)
print(df_train.shape)

(1458, 80)


In [6]:
#Log-transformation of the target variable
#Handle skewness(Reference - https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard)
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
df_train["SalePrice"] = np.log1p(df_train["SalePrice"])
df_train["SalePrice"].head()

0   12.248
1   12.109
2   12.317
3   11.849
4   12.429
Name: SalePrice, dtype: float64

In [7]:
ntrain = df_train.shape[0]
ntest = df_test.shape[0]
y_train_all = df_train.SalePrice.values
all_data = pd.concat((df_train, df_test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2917, 79)


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [8]:
def datapreprocessing(all_data):
    all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
    all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
    all_data["Alley"] = all_data["Alley"].fillna("None")
    all_data["Fence"] = all_data["Fence"].fillna("None")
    all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
    
    #Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
    all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
        all_data[col] = all_data[col].fillna('None')
        
    #Replacing missing data with 0 (Since No garage = no cars in such garage.)
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        all_data[col] = all_data[col].fillna(0)
        
    # missing values are likely zero for having no basement
    for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
        all_data[col] = all_data[col].fillna(0)
        
    # For all these categorical basement-related features, NaN means that there is no basement.
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        all_data[col] = all_data[col].fillna('None')
        
    #NA most likely means no masonry veneer for these houses. We can fill 0 for the area and None for the type.
    all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
    all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
    
    # 'RL' is by far the most common value. So we can fill in missing values with 'RL'
    all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
    
    #For this categorical feature all records are "AllPub", except for one "NoSeWa" and 2 NA . 
    #Since the house with 'NoSewa' is in the training set, this feature won't help in predictive modelling. 
    #We can then safely remove it.
    all_data = all_data.drop(['Utilities'], axis=1)
    
    #data description says NA means typical
    all_data["Functional"] = all_data["Functional"].fillna("Typ")
    
    #It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.
    all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
    
    #Only one NA value, and same as Electrical, we set 'TA' (which is the most frequent) for the missing value in KitchenQual.
    all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
    
    #Exterior 1 & 2 have only one missing value. We will just substitute in the most common string
    all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
    all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
    
    all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
    all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
    
    #Transforming some numerical variables that are really categorical

    #MSSubClass=The building class
    all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


    #Changing OverallCond into a categorical variable
    all_data['OverallCond'] = all_data['OverallCond'].astype(str)


    #Year and month sold are transformed into categorical features.
    all_data['YrSold'] = all_data['YrSold'].astype(str)
    all_data['MoSold'] = all_data['MoSold'].astype(str)
    
    return all_data
    
    
#datapreprocessing(df_train)
#datapreprocessing(df_test)
all_data = datapreprocessing(all_data)

#all_data.dtypes

In [9]:
from sklearn.preprocessing import LabelEncoder
def encodeCategories(all_data):
    cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
            'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
            'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
            'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 'YrSold', 'MoSold')
    # process columns, apply LabelEncoder to categorical features
    for c in cols:
        lbl = LabelEncoder() 
        lbl.fit(list(all_data[c].values)) 
        all_data[c] = lbl.transform(list(all_data[c].values))

    # shape        
    print('Shape all_data: {}'.format(all_data.shape))
    return all_data
    
#encodeCategories(df_train)
#encodeCategories(df_test)
all_data = encodeCategories(all_data)

Shape all_data: (2917, 78)


In [10]:
# Adding total sqfootage feature 
#df_train['TotalSF'] = df_train['TotalBsmtSF'] + df_train['1stFlrSF'] + df_train['2ndFlrSF']
#df_test['TotalSF'] = df_test['TotalBsmtSF'] + df_test['1stFlrSF'] + df_test['2ndFlrSF']
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [11]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

skewness_threshold = 0.75

def skewnesshandling(all_data):
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

    # Check the skew of all numerical features
    skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
    print("\nSkew in numerical features: \n")
    skewness = pd.DataFrame({'Skew' :skewed_feats})
    print(skewness.head(10))

    skewness = skewness[abs(skewness) > skewness_threshold]
    print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
    
    from scipy.special import boxcox1p
    skewed_features = skewness.index
    lam = 0.15
    for feat in skewed_features:
        #all_data[feat] += 1
        all_data[feat] = boxcox1p(all_data[feat], lam)
    return all_data
    
    #all_data[skewed_features] = np.log1p(all_data[skewed_features])
#skewnesshandling(df_train)
#skewnesshandling(df_test)
all_data = skewnesshandling(all_data)


Skew in numerical features: 

                Skew
MiscVal       21.940
PoolArea      17.689
LotArea       13.109
LowQualFinSF  12.085
3SsnPorch     11.372
LandSlope      4.973
KitchenAbvGr   4.301
BsmtFinSF2     4.145
EnclosedPorch  4.002
ScreenPorch    3.945
There are 59 skewed numerical features to Box Cox transform


In [12]:
# Separate numerical and categorical columns
def getNumericaldata(df,excludefield):
    quantitative = [f for f in df.columns if df.dtypes[f] != 'object']
    quantitative.remove(excludefield)
    return quantitative

def getCategoricaldata(df):
    qualitative = [f for f in df.columns if df.dtypes[f] == 'object']
    return qualitative

numcols = getNumericaldata(df_train,'SalePrice')
catcols = getCategoricaldata(df_train)
alldata_catcols = getCategoricaldata(all_data)

print(numcols)
print(catcols)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCon

In [13]:
def convertasCategory(df,cols):
    for col in cols:
        df[col] = df[col].astype('category')
        df[col+'_code'] = df[col].cat.codes
        df[col] = df[col +'_code']
        df.drop(labels=col+'_code', axis="columns", inplace=True)
    return df

if(USE_DUMMY_CAT_FEATURES):
    #df_train = pd.get_dummies(df_train)
    #print(df_train.shape)
    #df_test = pd.get_dummies(df_test)
    #print(df_test.shape)
    all_data = pd.get_dummies(all_data)
    print(all_data.shape)
else:
    all_data = convertasCategory(all_data,catcols)
    #convertasCategory(df_train,catcols)
    #convertasCategory(df_test,ccatcolsatcols)

(2917, 220)


In [14]:
   

#print(df_train.head())
#print(df_train.dtypes)
# Handle remaining missing values for numerical features by using median as replacement
#print("NAs for numerical features in train : " + str(df_train.isnull().values.sum()))
######df_train = df_train.fillna(df_train.median())
######df_test = df_test.fillna(df_train.median())
#print("Remaining NAs for numerical features in train : " + str(df_train.isnull().values.sum()))
#print("Remaining NAs for numerical features in test : " + str(df_test.isnull().values.sum()))
print("Remaining NAs for numerical features in all_data : " + str(all_data.isnull().values.sum()))

Remaining NAs for numerical features in all_data : 0


In [15]:
#dropcols=['BldgType_1Fam', 'Condition1_Artery', 'Condition2_Artery','Exterior1st_AsbShng','Exterior1st_CBlock',
#          'Exterior2nd_AsbShng','Foundation_BrkTil','GarageType_2Types','Heating_Floor','HouseStyle_1.5Fin',
#          'LandContour_Bnk','LotConfig_Corner','MSZoning_C (all)','MasVnrType_BrkCmn','MiscFeature_Gar2',
 #         'Neighborhood_Blmngtn','RoofMatl_ClyTile','RoofStyle_Flat','SaleCondition_Abnorml','SaleType_COD',
  #        'Utilities_AllPub','YearBuilt','YearRemodAdd','GarageYrBlt','TotalSF','GrLivArea','MiscFeature_None',
   #       'PoolQC','RoofMatl_CompShg','Heating_GasA','1stFlrSF','Electrical_SBrkr','Condition2_Norm','GarageArea',
    #      'RoofStyle_Gable','LotArea','Street','TotRmsAbvGrd','OverallQual','LotFrontage','KitchenAbvGr','TotalBsmtSF',
     #     'MSZoning_RL','Exterior1st_VinylSd','GarageType_Attchd','GarageCond']
dropcols=['YearRemodAdd', 'GrLivArea', '1stFlrSF', 'GarageArea', 'YearBuilt', 'TotRmsAbvGrd', 'LotFrontage',
          'OverallQual', 'LotArea', 'TotalBsmtSF', 'KitchenAbvGr', 'GarageYrBlt', 'BedroomAbvGr', 'OverallCond', 
          'FullBath', 'GarageCars', 'MoSold', 'BsmtUnfSF']
dropcols=['Exterior2nd_CmentBd','GarageYrBlt','Exterior2nd_MetalSd','Exterior2nd_VinylSd','SaleType_New']#dropping cols based on pandas profiling
all_data = all_data.drop(dropcols,axis=1)
print(all_data.shape)

(2917, 215)


In [16]:
import pandas_profiling
#pandas_profiling.ProfileReport(all_data)

In [17]:
df_train_clean = all_data[:ntrain]
df_test_clean = all_data[ntrain:]
print(df_train_clean.shape)
print(df_test_clean.shape)

display(df_train_clean.head())
display(df_test_clean.head())

(1458, 215)
(1459, 215)


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_Oth,SaleType_WD
0,11.693,11.686,0.000,0.730,1.541,1.820,1.541,11.170,0.000,1.194,...,1,0,0,0,0,0,0,0,0,1
1,12.792,0.000,0.000,0.730,1.541,1.820,0.730,12.063,0.000,0.000,...,1,0,0,0,0,0,0,0,0,1
2,11.892,11.725,0.000,0.730,1.541,1.820,1.194,10.200,0.000,1.194,...,1,0,0,0,0,0,0,0,0,1
3,12.014,11.354,0.000,0.730,1.541,0.730,1.541,8.274,0.000,0.000,...,0,0,0,0,0,0,0,0,0,1
4,12.511,12.271,0.000,0.730,1.820,1.820,0.000,10.971,0.000,1.194,...,1,0,0,0,0,0,0,0,0,1


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_Oth,SaleType_WD
1458,11.819,0.000,0.000,0.730,1.194,1.820,1.541,10.105,7.397,2.056,...,1,0,0,0,0,0,0,0,0,1
1459,12.944,0.000,0.000,0.730,1.541,1.820,1.541,11.901,0.000,0.000,...,1,0,0,0,0,0,0,0,0,1
1460,11.916,11.151,0.000,0.730,1.541,1.820,1.541,11.477,0.000,1.194,...,1,0,0,0,0,0,0,0,0,1
1461,11.910,11.063,0.000,0.730,1.541,1.820,1.541,10.750,0.000,1.194,...,1,0,0,0,0,0,0,0,0,1
1462,12.834,0.000,0.000,0.730,1.194,1.820,1.541,8.720,0.000,0.000,...,1,0,0,0,0,0,0,0,0,1


In [18]:
df_train_clean.to_csv('./data/train_X_clean.csv',index=False)
df_test_clean.to_csv('./data/test_X_clean.csv',index=False)

dataset = pd.DataFrame({'SalePrice': y_train_all})
dataset.to_csv('./data/train_y_clean.csv',index=False)
#print(dataset)
#np.savetxt("./data/train_y_clean.csv", y_train_all, delimiter=",")

In [19]:
print(y_train_all)

[12.24769912 12.10901644 12.31717117 ... 12.49313327 11.86446927
 11.90159023]
